# Deep Learning Models

This is not a deep learning course, but by request, I will introduce simple examples to get started in deep learning and highlight some aspects that I find interesting. In particular, strategies for optimization and specialized architectures are not covered.

In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
import tensorflow_datasets as tfds

## Building models

### Prepare data

In [ ]:
ds, info = tfds.load(name='iris', 
                     as_supervised=True, 
                     with_info=True)

In [ ]:
for row, label in ds['train'].take(3):
    print(row.numpy(), label.numpy())

### Pre-process data

If you need to pre-process your data, see https://keras.io/api/preprocessing/. We will not do any pre-processing for simplicity.

### Sequential API

If the entire pipeline is a single chain of layers, the Sequential API is the simplest to use.

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input

#### Set random seed for reproducibility

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
model = Sequential()
model.add(Dense(8, input_shape=(4,)))
model.add(Dense(4, activation='relu'))
model.add(Dense(3, activation='softmax'))

**Alternative model specification**

```python
model = Sequential(
    Dense(8, input_shape=(4,)),
    Dense(4, activation='relu'),
    Dense(3, activation='softmax')
)
```

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
hist = model.fit(ds['train'].batch(16), epochs=100, verbose=0)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12, 4))
for ax, measure in zip(axes, ['loss', 'accuracy']):
    ax.plot(hist.history[measure], label=measure)
    ax.legend()
pass

### Functional API

The functional API provides flexibility, allowing you to build models with multiple inputs, multiple outputs, or branch and merge architectures.

#### Set random seed for reproducibility

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
input = Input(shape=(4,))
x = Dense(8)(input)
x = Dense(4, activation='relu')(x)
output = Dense(3, activation='softmax')(x)
model = Model(inputs=[input], outputs=[output])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
hist = model.fit(ds['train'].batch(16), epochs=100, verbose=0)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12, 4))
for ax, measure in zip(axes, ['loss', 'accuracy']):
    ax.plot(hist.history[measure], label=measure)
    ax.legend()
pass

## Built-in models and transfer learning

Keras comes with many built-in model architectures that serve as great starting points. Sometimes they even come pre-trained on massive data sets and can be used out-of-the-box, or improved with some transfer learning.

In [ ]:
ds, info = tfds.load(name='fashion_mnist', 
                     as_supervised=True, 
                     batch_size=32,
                     with_info=True)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

#### Preprocesss the image to make it work InceptionV3

In [ ]:
size = [75, 75]
X_train = (ds['train'].
           map(lambda image, label: 
               (tf.image.resize(tf.image.grayscale_to_rgb(image), size), label)))
X_test = (ds['test'].
          map(lambda image, label:
              (tf.image.resize(tf.image.grayscale_to_rgb(image), size), label)))

In [ ]:
shape = [item[0].shape for item in X_train.take(1)][0]
shape

In [ ]:
base_model = InceptionV3(
    input_shape=shape[1:],
    weights='imagenet', 
    include_top=False,
    classes=10
)

#### Freeze weights in base model

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

#### Add the last layers of the model to predict 10 classes

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(X_train, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(X_test)
test_acc

## Custom classes

### Custom loss functions

### Custom initializers

### Custom optimizers

### Custom activation function

## Generative deep learning with autoencoders

### Vanilla autencoders

### Variattional autoencoders

## Interpretable deep learning

## Model agnostic visualization

## Hyperparameter optimization

## Probabilistic deep learning

### Generating distributions as outputs

### Monte Carlo dropout

### Bayesian nets